# DATA CLEANING 

# fns to import statements

In [96]:
# imports 
import pandas as pd
import numpy as np 
import yfinance as yf 
from yahoofinancials import YahooFinancials

def getIncomeStatement(ticker, yahoo_financials):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler income statement for the company
    """
    # import data
    # yahoo_financials = YahooFinancials('AAPL')
    data = yahoo_financials.get_financial_stmts('quarterly', 'income')
    # Drill down in dictionary to get the data we want 
    dict_list = data['incomeStatementHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getBalanceSheet(ticker, yahoo_financials):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler balance for the company 
    """
    # import data
    # yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_financial_stmts('quarterly', 'balance')
    # print(data)
    # Drill down in dictionary to get the data we want 
    dict_list = data['balanceSheetHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getCashSheet(ticker, yahoo_financials):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler balance for the company 
    """
    # import data
    # yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_financial_stmts('quarterly', 'cash')
    # Drill down in dictionary to get the data we want 
    dict_list = data['cashflowStatementHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getPrices(ticker, yahoo_financials):
    """
    Inputs: ticker of a company 
    Outputs: the prices of the 4 quarterly statements for 2022 
    NOTE: price is technically for 12/30 bc there was no price available for 12/31
    """
    # yahoo_financials = YahooFinancials(ticker)
    lst_dates_interst = ['2022-03-31', '2022-06-30','2022-09-30', '2022-12-30']
    lst_end_dates = ['2022-4-1', '2022-7-1', '2022-10-1', '2023-1-2']

    # get first data so that it can be concated later on 
    data = yahoo_financials.get_historical_price_data(start_date=lst_dates_interst[0], end_date=lst_end_dates[0], time_interval='daily')
    df = pd.DataFrame(data[ticker]['prices'])

    # go through and get the prices for each data and add it to the df
    for i in range(len(lst_dates_interst)-1):
        data = yahoo_financials.get_historical_price_data(start_date=lst_dates_interst[i+1], end_date=lst_end_dates[i+1], time_interval='daily')
        temp = pd.DataFrame(data[ticker]['prices'])
        df = pd.concat([df, temp], axis=0)
    # Set dates to be axis. Note price is technically for 12/30 bc there was no price available for 12/31
    df = df.set_axis(['2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31'], axis='index')

    # get rid of date columns because they're now the index
    df = df.drop(columns=['formatted_date', 'date'], axis=0)
    return df

# fns to extract metrics of interest

In [97]:
# create functions that extract metrics 

def clean_df(df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that has actually column names and gets rid of unneccsary columns
    NOTE: this function is called by each of the three function below before extracting metrics.  
    """
    cols = df.iloc[0]
    df = df.rename(columns=cols)
    df = df.drop(['date'])
    return df

def getIncomeMetrics (df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    income_metrics = [
        'dilutedNIAvailtoComStockholders',
        'grossProfit',
        'netIncome',
        'totalExpenses',
        'totalRevenue',
        'basicEPS', 
    ]
    df2 = pd.DataFrame([])
    for metric in income_metrics:
        if metric in df.columns:
            # get the data from df and put it in the correct column in df2
            df2[metric] = df[metric]
        else:
            # fill the data with NA
            df2[metric] = pd.NA
    return df2

def getBalanceSheetMetrics (df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    balance_metrics = [
    'cashEquivalents',
    'currentAssets',
    'currentDebt',
    'netDebt',
    'totalCapitalization',
    'totalDebt',
    'totalAssets',
    'tangibleBookValue',
    'shareIssued'
    ]
    df2 = pd.DataFrame([])
    for metric in balance_metrics:
        if metric in df.columns:
            # get the data from df and put it in the correct column in df2
            df2[metric] = df[metric]
        else:
            # fill the data with NA
            df2[metric] = pd.NA
    return df2

def getCashMetrics (df): 
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    cash_metrics = [
    'freeCashFlow',
    ]
    df2 = pd.DataFrame([])
    for metric in cash_metrics:
        if metric in df.columns:
            # get the data from df and put it in the correct column in df2
            df2[metric] = df[metric]
        else:
            # fill the data with NA
            df2[metric] = pd.NA
    return df2

# fns to Calculate Metrics 

In [98]:
def marketCap (price, sharesIssued):
    return price * sharesIssued

def pricePerEarningsPerShare (price, eps):
    return price/eps

def priceToBook (price, netAssets):
    return price / netAssets

In [99]:
def getCalculatedMetrics(df):
    """
    Inputs: Takes a df 
    Outputs: The same dataframe with columns added that contian the metrics. 
    """
    # empty lists to store the metrics 
    temp_market, temp_price_per_eps, temp_price_to_book =  [], [], []

    # go through each row of the datafrane (this may be too slow once we do multiple companies )
    for i,v in df.iterrows():
        
        temp_market.append(marketCap(v['open'], v['shareIssued'])) 
        temp_price_per_eps.append(pricePerEarningsPerShare(v['open'], v['basicEPS']))
        temp_price_to_book.append(priceToBook(v['open'], v['netDebt']))
    df.insert(loc = len(df.columns), column='marketCap', value=temp_market)
    df.insert(loc = len(df.columns), column='priceToEarningsRatio', value=temp_price_per_eps)
    df.insert(loc = len(df.columns), column='priceToBook', value=temp_price_to_book)
    return df

# fn to produce dataframe for a company

In [100]:
from yahoofinancials import YahooFinancials

def getDfForCompany(ticker):
    """
    Inputs: ticker of compnay 
    Outputs: A dataframe that contains all the metrics of interest for this project that come from financial statements and price history (ie non calculated metrics)
    """

    yahoo_financials = YahooFinancials(ticker)


    df = getIncomeStatement(ticker, yahoo_financials)
    inc_df = getIncomeMetrics(pd.DataFrame(df))

    df = getBalanceSheet(ticker, yahoo_financials)
    bal_df = getBalanceSheetMetrics(df)

    df = getCashSheet(ticker, yahoo_financials)
    cash_df = getCashMetrics(df)

    price_df = getPrices(ticker, yahoo_financials)

    # merge dataframes
    merge = pd.concat([inc_df, bal_df, cash_df, price_df], axis=1)

    # add company ticker to each quarter
    merge["Company"] = [ticker for i in range(4)]

    # calculate metrics 
    merge = getCalculatedMetrics(merge)
    return merge


In [101]:
# import time
# start = time.time()
# df = getDfForCompany('GOOG')
# end = time.time()
# print("Function takes ", end-start, "seconds") # ~19.9s as of 4/6 -> 18.79s, 18.27s, 19.4, 18.2 after changing the fns. 
# df

Function takes  18.183467149734497 seconds


,dilutedNIAvailtoComStockholders,grossProfit,netIncome,totalExpenses,totalRevenue,basicEPS,cashEquivalents,currentAssets,currentDebt,netDebt,...,high,low,open,close,volume,adjclose,Company,marketCap,priceToEarningsRatio,priceToBook
2022-03-31,16436000000.0,38412000000.0,16436000000.0,47917000000.0,68011000000.0,1.245001,NaN,177853000000.0,<NA>,<NA>,...,142.644501,139.619003,142.448502,139.649506,29516000,139.649506,GOOG,1.876796e+12,114.416375,<NA>
2022-06-30,17074000000.0,39581000000.0,16002000000.0,50232000000.0,69685000000.0,1.220001,NaN,172371000000.0,<NA>,<NA>,...,111.329803,107.309998,110.499496,109.372498,38046000,109.372498,GOOG,1.445112e+12,90.573284,<NA>
2022-09-30,13910000000.0,37934000000.0,13910000000.0,51957000000.0,69092000000.0,1.07,NaN,166109000000.0,<NA>,<NA>,...,99.494003,96.029999,97.730003,96.150002,26277800,96.150002,GOOG,1.267656e+12,91.336452,<NA>
2022-12-31,59972000000.0,40706000000.0,13624000000.0,207994000000.0,76048000000.0,1.06,NaN,164795000000.0,<NA>,<NA>,...,88.830002,87.029999,87.364998,88.730003,19190300,88.730003,GOOG,1.122553e+12,82.419809,<NA>


# Test with multiple companies

In [102]:
import pandas as pd
import time 
start = time.time()
tickers = ['AAPL', 'AMZN', 'AAL']
df = getDfForCompany(tickers[0])
tickers.remove(tickers[0])
for ticker in tickers:
    df = pd.concat(df, getDfForCompany(ticker))
end = time.time()
print("Function takes ", end-start, "seconds") # 
df

In [103]:
# yahoo_financials = YahooFinancials("GOOG")

# df = getBalanceSheet("GOOG", yahoo_financials)

# df = clean_df(df)
# balance_metrics = [
# 'cashEquivalents',
# 'currentAssets',
# 'currentDebt',
# 'netDebt',
# 'totalCapitalization',
# 'totalDebt',
# 'totalAssets',
# 'tangibleBookValue',
# 'shareIssued'
# ]
# df2 = pd.DataFrame([])